In [0]:
import time
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
torch.cuda.empty_cache()

In [0]:
with open('/content/drive/My Drive/fsog.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [0]:
print(text[:665])

Chapter One 

I scowl with frustration at myself in the mirror. Damn my hair - it just won’t behave, 
and damn Katherine Kavanagh for being ill and subjecting me to this ordeal. I should be 
studying for my final exams, which are next week, yet here I am trying to brush my hair 
into submission. I must not sleep with it wet. I must not sleep with it wet. Reciting this 
mantra several times, I attempt, once more, to bring it under control with the brush. I roll 
my eyes in exasperation and gaze at the pale, brown-haired girl with blue eyes too big for 
her face staring back at me, and give up. My only option is to restrain my wayward hair in 
a ponytail and 


In [0]:
unique_characters = set(text)
print(f"Number of unique characters: {unique_characters}\n\nLength: {len(unique_characters)}")

Number of unique characters: {'h', ' ', 'v', '?', 'm', 'P', ')', 'e', '!', 'Z', '&', '1', '+', '”', 'j', 'q', ',', '^', 'Q', 'R', ']', 'z', 'n', '6', 'U', 'A', 'p', '-', 'a', '©', 'g', '2', 'H', '9', '—', 't', '$', '(', ':', 'M', 'f', '0', 'C', 'r', 'i', '|', '7', 'o', '\n', 'y', '>', 'E', 'D', 'B', '~', 'Y', 'u', 'F', 'k', '/', '_', '*', 'W', '8', '5', 'J', '[', 'w', '.', 'l', 'x', 'N', 'T', '“', 'O', '’', '"', '\\', "'", 'd', 'K', 'X', '4', 'V', 'L', 'c', 's', 'I', ';', 'b', '3', '■', '‘', '•', 'G', 'S'}

Length: 96


In [0]:
decoder = dict(enumerate(unique_characters))

In [0]:
encoder = {char: index for index, char in decoder.items()}

In [0]:
encoded_text = np.array([encoder[char] for char in text])

In [0]:
print(encoded_text[:500])

[42  0 28 26 35  7 43  1 74 22  7  1 48 48 87  1 86 85 47 67 69  1 67 44
 35  0  1 40 43 56 86 35 43 28 35 44 47 22  1 28 35  1  4 49 86  7 69 40
  1 44 22  1 35  0  7  1  4 44 43 43 47 43 68  1 52 28  4 22  1  4 49  1
  0 28 44 43  1 27  1 44 35  1 14 56 86 35  1 67 47 22 75 35  1 89  7  0
 28  2  7 16  1 48 28 22 79  1 79 28  4 22  1 80 28 35  0  7 43 44 22  7
  1 80 28  2 28 22 28 30  0  1 40 47 43  1 89  7 44 22 30  1 44 69 69  1
 28 22 79  1 86 56 89 14  7 85 35 44 22 30  1  4  7  1 35 47  1 35  0 44
 86  1 47 43 79  7 28 69 68  1 87  1 86  0 47 56 69 79  1 89  7  1 48 86
 35 56 79 49 44 22 30  1 40 47 43  1  4 49  1 40 44 22 28 69  1  7 70 28
  4 86 16  1 67  0 44 85  0  1 28 43  7  1 22  7 70 35  1 67  7  7 58 16
  1 49  7 35  1  0  7 43  7  1 87  1 28  4  1 35 43 49 44 22 30  1 35 47
  1 89 43 56 86  0  1  4 49  1  0 28 44 43  1 48 44 22 35 47  1 86 56 89
  4 44 86 86 44 47 22 68  1 87  1  4 56 86 35  1 22 47 35  1 86 69  7  7
 26  1 67 44 35  0  1 44 35  1 67  7 35 68  1 87  1

In [0]:
with open('/content/drive/My Drive/fsog_decoder.dat', 'wb') as f:
  pickle.dump(decoder, f)

In [0]:
with open('/content/drive/My Drive/fsog_encoder.dat', 'wb') as f:
  pickle.dump(encoder, f)

In [0]:
def one_hot_encoder(encoded_text, nunique):
    one_hot = np.zeros((encoded_text.size, nunique))
    one_hot = one_hot.astype(np.float32)
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
    one_hot = one_hot.reshape((*encoded_text.shape, nunique))
    return one_hot

In [0]:
def create_batches(encoded_text, batch_size=10, sequence_length=50):
    num_chars = batch_size * sequence_length
    num_batches = int(len(encoded_text)/num_chars)
    
    encoded_text = encoded_text[:num_batches*num_chars]
    
    encoded_text = encoded_text.reshape((batch_size, -1))
    
    for n in range(0, encoded_text.shape[1], sequence_length):
        x = encoded_text[:, n:n+sequence_length]
        y = np.zeros_like(x)
        
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, n+sequence_length]
        except:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, 0]
        
        yield x, y 

In [0]:
class GenerativeModel(nn.Module):
    def __init__(self, total_chars, hidden_size=256, num_layers=4, p=0.5, cuda=False):
        super().__init__()
        self.p = p
        self.num_layers= num_layers
        self.hidden_size = hidden_size
        self.use_cuda = cuda
        
        self.total_chars = total_chars
        self.decoder = dict(enumerate(self.total_chars))
        self.encoder = {char: index for index, char in self.decoder.items()}
        
        self.char_len = len(self.total_chars)
        
        self.lstm = nn.LSTM(self.char_len, self.hidden_size, self.num_layers, dropout=p, batch_first=True)
        
        self.dropout = nn.Dropout(p)
        
        self.fc1 = nn.Linear(self.hidden_size, self.char_len)
    
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        
        out = self.dropout(out)
        
        out = out.contiguous().view(-1, self.hidden_size)
        
        out = self.fc1(out)
        
        return out, hidden
    
    def hidden_state(self, batch_size):
        if self.use_cuda:
            hidden = (torch.zeros(self.num_layers, batch_size, self.hidden_size).cuda(), torch.zeros(self.num_layers, batch_size, self.hidden_size).cuda())
        
        else:
            hidden = (torch.zeros(self.num_layers, batch_size, self.hidden_size), torch.zeros(self.num_layers, batch_size, self.hidden_size))

        return hidden    

In [0]:
model = GenerativeModel(total_chars=unique_characters, hidden_size=512, num_layers=4, p=0.4, cuda=True)

In [0]:
train_percent = 0.9
train_index = int(len(encoded_text) * train_percent)
train_data = encoded_text[:train_index]
test_data = encoded_text[train_index:]

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
criterion = nn.CrossEntropyLoss().cuda()
epochs = 60
batch_size = 64
sequence_length = 100
num_char = max(encoded_text) + 1
t = 0 

In [0]:
model.train()

if model.use_cuda:
    model = model.cuda()

start_time = time.time()

for i in range(epochs):
    e_start = time.time()
    
    hidden = model.hidden_state(batch_size)
    
    for x, y in create_batches(train_data, batch_size, sequence_length):
        t += 1
        
        x = one_hot_encoder(x, num_char)
        
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)
        
        if model.use_cuda:
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        hidden = tuple([state.data for state in hidden])
        
        optimizer.zero_grad()
        
        lstm_output, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_output, targets.view(batch_size*sequence_length).long())
        
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        
        optimizer.step()
        
        if t % 50 == 0:
            e_end = time.time() - e_start
            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()
            
            for x, y in create_batches(test_data, batch_size, sequence_length):
                x = one_hot_encoder(x, num_char)
        
                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                if model.use_cuda:
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                    
                val_hidden = tuple([state.data for state in hidden])
                
                lstm_out, val_hidden = model.forward(inputs, val_hidden)
                val_loss = criterion(lstm_out, targets.view(batch_size * sequence_length).long())
                val_losses.append(val_loss.item())
            
            model.train()
            
            print(f"Epoch {i+1}\nLoss: {loss.item():.4f} | Validation Loss: {val_loss.item():.4f} | Duration: {e_end/60:.2f} minutes")
           
end_time = time.time() - start_time
print(f"\nTotal Training Duration {end_time/60:.2f}")

Epoch 1
Loss: 3.2173 | Validation Loss: 3.1673 | Duration: 0.12 minutes
Epoch 1
Loss: 3.2407 | Validation Loss: 3.1682 | Duration: 0.24 minutes
Epoch 2
Loss: 3.1947 | Validation Loss: 3.1661 | Duration: 0.07 minutes
Epoch 2
Loss: 3.2041 | Validation Loss: 3.1664 | Duration: 0.20 minutes
Epoch 3
Loss: 3.1985 | Validation Loss: 3.1655 | Duration: 0.03 minutes
Epoch 3
Loss: 3.1992 | Validation Loss: 3.1650 | Duration: 0.16 minutes
Epoch 3
Loss: 3.1422 | Validation Loss: 3.1638 | Duration: 0.28 minutes
Epoch 4
Loss: 3.1520 | Validation Loss: 3.1623 | Duration: 0.11 minutes
Epoch 4
Loss: 3.1773 | Validation Loss: 3.1453 | Duration: 0.23 minutes
Epoch 5
Loss: 2.8405 | Validation Loss: 2.7904 | Duration: 0.07 minutes
Epoch 5
Loss: 2.6333 | Validation Loss: 2.6049 | Duration: 0.19 minutes
Epoch 6
Loss: 2.4961 | Validation Loss: 2.4431 | Duration: 0.02 minutes
Epoch 6
Loss: 2.3648 | Validation Loss: 2.3438 | Duration: 0.15 minutes
Epoch 6
Loss: 2.3236 | Validation Loss: 2.2810 | Duration: 0.27 

In [0]:
torch.save(model.state_dict(), '/content/drive/My Drive/fsog.pt')

In [0]:
with open('/content/drive/My Drive/fsog_model_decoder.dat', 'wb') as f:
  pickle.dump(model.decoder, f)

In [0]:
with open('/content/drive/My Drive/fsog_model_encoder.dat', 'wb') as f:
  pickle.dump(model.encoder, f)

In [0]:
def get_next_char(model, char, hidden=None, k=1):
    encoded_text = model.encoder[char]
    encoded_text = np.array([[encoded_text]])
    encoded_text = one_hot_encoder(encoded_text, len(model.total_chars))
    inputs = torch.from_numpy(encoded_text)
    
    if model.use_cuda:
        inputs = inputs.cuda()
    
    hidden = tuple([state.data for state in hidden])
    
    out, hidden = model(inputs, hidden)
    
    probs = F.softmax(out, dim=1).data
    
    if model.use_cuda:
        probs = probs.cpu()
        
    probs, index_pos = probs.topk(k)
    
    index_pos = index_pos.numpy().squeeze()
    
    probs = probs.numpy().flatten()
    
    probs = probs/probs.sum()
    
    next_char = np.random.choice(index_pos, p=probs)
    
    return model.decoder[next_char], hidden

In [0]:
def generate_text(model, size, seed='I', k=1):
    if model.use_cuda:
        model = model.cuda()
    else:
        model = model.cpu()
    
    model.eval()
    
    output_chars = [c for c in seed]
    
    hidden = model.hidden_state(1)
    
    for char in seed:
        char, hidden = get_next_char(model, char, hidden, k=k)
        
    output_chars.append(char)
    
    for i in range(size):
        char, hidden = get_next_char(model, output_chars[-1], hidden, k=k)
        output_chars.append(char)
    
    return ''.join(output_chars)

In [0]:
print(generate_text(model, 500, seed='I', k=3))

I he’s not sure I’m 
touching my hand and then softly. I shake my hand, and I can’t tell her. I want to stay, 
and I close my helicopter that the words, and I can’t hold my exaspears. I am not touched 
to many move, and he still draps my hand. He stares at me. “Well, this is 
shall with that about my stomach. I don’t want to say anything, and I’m 
too with that. Tomorrow and the moment,” I mutter. His tone is 
stranged and grasps my hand. 

“I’ve not going to stay.” He presses his explession in my


In [0]:
print(generate_text(model, 500, seed='kill', k=10))

kille what after I sit. It 
feels awoins from Christian caulot to know my head of times. I don’t want to hide. I 
don’t sit will be with me. Why he’ll have boter in his feet to some our arrungered. Come in a 
sweet talling thought makes them thrush between answer at shail of the fitting of trush with hard. I 
could say what’s not as is a deliberately. I scill one switch, and soft, and I don’t understand. 

He smiles about his tea and holding me suddenly. He’s still half to hard in the corred? 


“Wea


In [0]:
print(generate_text(model, 500, seed='like', k=40))

like, 
detjo-eting magor. I expressed in the right of. Add I’m close into my 
cboth - wink? Just only the onay under wime concertation to he - pleasen’t 
speen things to a push and clampes. I build night. 

“No.” I release my resiler thoughts through his ear unleashed, catical comfletely back to any 
long. 

“I’ll tell me what pushed it de. She’s horrided by your. My mother is really this yele. Maybe they’re company, then we’re entrackedly to 
trust me.” 

I hang as a very behind. You like the doow. 


In [0]:
print(generate_text(model, 1000, seed='pleasure', k=50))

pleasure? 

“Well, what I would like you, piils. Kate two start had away for you?” 

“You can tell him?” He smiles. 

“You later him?” I whisper. My skyterces of me But an undo- 
small moments there. I’m never seing to me, door, the soft of suctisualy moun-sudy 
forsal clamp is in My breath. She says her’s like this. Them want his things lot other 
this wait deep, and the long, books in decresting appolrant of there and 
the seven mine, but she leans down on between his phustration, staring from me. My thunging 
from the pursing confuser, and Christian pives crapily. Beens for a little fire he is he like that? 

“Ahais the time we’re love. So impecialAs? And about Paris, and I don’t think that 
only ne madness with you,” he says from the progeded. Bending, I shall 
whise this intitated sound. I recover to ask the kitchen, learing out of the arms and talking 
on the elevator elevant? She holds the black water, on all a smile, and 
not litten to reach my face. 


Christian and hesset the

In [0]:
print(generate_text(model, 10, seed='passion', k=50))

passion he squeeze


In [0]:
print(generate_text(model, 10, seed='stop', k=50))

stopten, I ging
